## Building A Chatbot
In this module We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [63]:
import os
from dotenv import load_dotenv
load_dotenv() ## Loading all the environment variable

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [64]:
import os
print(os.getenv("GROQ_API_KEY"))  # Should print your key if set correctly


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


In [65]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A790A77980>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A7909B7AD0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [66]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Hargovind Ahirwar and I am a AI engineer")])

AIMessage(content="Hello Hargovind Ahirwar! It's nice to meet you.  \n\nThat's great that you're an AI engineer. It's a fascinating field with so much potential. What kind of work do you do in AI? Are you working on any exciting projects right now? \n\nI'm always eager to learn more about what people are doing in the world of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 26, 'total_tokens': 111, 'completion_time': 0.154545455, 'prompt_time': 0.005126689, 'queue_time': 0.75552129, 'total_time': 0.159672144}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--7b17e822-bab2-42f5-9bda-49bdb1735761-0', usage_metadata={'input_tokens': 26, 'output_tokens': 85, 'total_tokens': 111})

In [67]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi ,My name is Hargovind Ahirwar and I am a AI enginee"),
    
    HumanMessage(content="hey What 's my name and what do i do?")
])

AIMessage(content="You told me your name is Hargovind Ahirwar and that you are an AI engineer.  \n\nIs there anything else you'd like to talk about? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 49, 'total_tokens': 88, 'completion_time': 0.070909091, 'prompt_time': 0.008510882, 'queue_time': 0.412936888, 'total_time': 0.079419973}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--a15dc068-0f27-4858-af43-44309465a9c6-0', usage_metadata={'input_tokens': 49, 'output_tokens': 39, 'total_tokens': 88})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [68]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [69]:
config = {"configurable":{"session_id":"chat1"}}

In [71]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi , My name is Hargovind Ahirwar and I am a AI Engineer")]},
    config=config
)


In [72]:
response.content

"Hi Hargovind Ahirwar, it's great to meet you!\n\nWhat kind of AI engineering are you involved in?  Do you specialize in a particular area like machine learning, natural language processing, computer vision, or something else? \n\nI'm always curious to learn about the different applications and challenges faced by AI engineers.\n"

In [73]:
response2 = with_message_history.invoke([
    HumanMessage(content="What is my name?")
],config= config)

response2

AIMessage(content='Your name is Hargovind Ahirwar.  😊  \n\nI remembered it from our earlier conversation!  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 202, 'total_tokens': 239, 'completion_time': 0.067272727, 'prompt_time': 0.012692253, 'queue_time': 0.243616866, 'total_time': 0.07996498}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c695c17b-6010-46a5-9777-5d4817a1b6ef-0', usage_metadata={'input_tokens': 202, 'output_tokens': 37, 'total_tokens': 239})

In [74]:
config2 = {"configurable":{"session_id":"chat2"}
}
response3 = with_message_history.invoke([
    HumanMessage(content="What you can do")
],config=config2)

response3


AIMessage(content="As an open-weights AI assistant, here are some of the things I can do:\n\n**Communication and Language:**\n\n* **Generate creative content:** Write stories, poems, articles, dialogue, and more.\n* **Translate languages:** Translate text between different languages.\n* **Summarize text:** Provide concise summaries of longer pieces of writing.\n* **Answer questions:** Answer your questions based on the information I have been trained on.\n* **Engage in conversation:** Have a natural-sounding conversation on a variety of topics.\n\n**Code and Data:**\n\n* **Generate code:** Write code in multiple programming languages.\n* **Explain code:** Provide explanations and comments for existing code.\n* **Identify patterns in data:** Analyze data and identify trends or patterns.\n\n**Other Capabilities:**\n\n* **Brainstorm ideas:** Help you generate new ideas for projects or problems.\n* **Provide definitions and explanations:** Define words and concepts.\n* **Create lists and o

In [75]:
response = with_message_history.invoke([
    HumanMessage(content="Hey , my name is Ajay and I am passionate AI engineer . My father is owner of Alifera construction")
    
],config=config2)

In [76]:
response = with_message_history.invoke([
    HumanMessage(content="Hey , what you know  about me? ")
    
],config=config2)

In [77]:
response.content

"Based on our conversation so far, I know that:\n\n* Your name is Ajay.\n* You're passionate about AI engineering.\n* Your father owns a construction company called Alifera.\n\nIs there anything else you'd like me to know about you? I'm always eager to learn more! 😊  \n\n\n\n\n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [78]:
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system","Ypu are a helpful assistant . Answer all the question to the best of your ablilty"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [79]:
chain.invoke({"messages" : [HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hi Krish! It's nice to meet you.\n\nI'm happy to help with any questions you have.  Just ask away! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 33, 'total_tokens': 67, 'completion_time': 0.061818182, 'prompt_time': 0.002363786, 'queue_time': 0.246693644, 'total_time': 0.064181968}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f2966de1-d600-48a2-9319-541865b2266d-0', usage_metadata={'input_tokens': 33, 'output_tokens': 34, 'total_tokens': 67})

In [80]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [81]:
config4 = {"configurable": {"session_id" : "chat3"}}
response = with_message_history.invoke(
    [
        HumanMessage(content = "Hi My name is Hargovind Ahirwar")
    ],
    config=config4
)
response

AIMessage(content="Hello Hargovind Ahirwar! It's nice to meet you. \n\nPlease ask me anything! I'll do my best to help. 😊  What can I do for you today? \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 38, 'total_tokens': 84, 'completion_time': 0.083636364, 'prompt_time': 0.002344785, 'queue_time': 0.244261638, 'total_time': 0.085981149}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--2970b2a1-c01f-4478-9374-a3cf3f2303f3-0', usage_metadata={'input_tokens': 38, 'output_tokens': 46, 'total_tokens': 84})

In [82]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"Your name is Hargovind Ahirwar!  \n\nI'm designed to remember information from our conversations. \n\nIs there something specific you'd like to talk about, Hargovind?\n"

In [83]:
## Add more complexity 
prompt = ChatPromptTemplate.from_messages([
    (
        "system","You are a helpful assistant. Answer all questions to the best of your ability in{language}.",),
    MessagesPlaceholder(variable_name="messages"),
    
])
chain = prompt | model

response = chain.invoke({
    "messages":[HumanMessage(content="hi my name is Hargovind")] , "language":"hindi"
})
response.content

In [84]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [85]:
with_messages_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [86]:
config = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    
        {'messages':[HumanMessage(content="Hi , my name is hargovind")],"language":"Hindi"},
        config=config
    
)
response.content

'नमस्ते हर्गोविंद! मुझे खुशी है कि आपसे मिलने को मिला। \n\nआप मुझसे क्या जानना चाहते हैं? मैं आपकी मदद करने के लिए तैयार हूँ। 😊\n'

In [87]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [88]:
response.content

'आपका नाम हर्गोविंद है।  😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [91]:
from langchain_core.messages import SystemMessage , trim_messages
trimmer = trim_messages(
    max_tokens=45,
    strategy= "last",
    token_counter = model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="Hi ! I'm Ajay"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what is 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="thanks"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [92]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")| trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages":messages+[HumanMessage(content="what ice cream do i like")],
        "language":"Hindi"
    }
)

response.content

'मुझे नहीं पता कि आपको किस तरह का आइसक्रीम पसंद है! मुझे आपकी पसंद के बारे में बताएं ताकि मैं आपको बेहतर सहायता कर सकूं.  क्या आपके पसंदीदा आइसक्रीम फ्लेवर हैं? 😋🍦\n'

In [94]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "Hindi",
    }
)
response.content

'तुमने 2 + 2 का सवाल पूछा था. \n'

In [95]:
## Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable":{"session_id":"chat5"}
}

In [96]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nWould you like to tell me your name? 😊  \n\n"

In [97]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "French",
    },
    config=config,
)

response.content

"Je suis désolé, mais je n'ai pas de mémoire des conversations passées. Pourrais-tu me reformuler ta question ? 😊  \n\n"